# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, observables: List[str] = []) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negative.
 |   

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,652.564610,0.694440,1234,653.259050,651.870170,1306.518099,1303.740339
6,380.612623,0.009923,1234,380.622546,380.602700,761.245092,761.205400
7,543.162775,-0.353520,1234,542.809255,543.516295,1085.618511,1087.032590
8,896.618869,-0.297303,1234,896.321566,896.916172,1792.643131,1793.832344
9,131.757963,0.786541,1234,132.544504,130.971422,265.089008,261.942845
10,15.918097,0.332964,1234,16.251062,15.585133,32.502123,31.170266
11,164.925526,-0.353543,1234,164.571983,165.279069,329.143966,330.558139
12,618.397286,-0.285240,1234,618.112045,618.682526,1236.224091,1237.365051
13,599.887467,0.200363,1234,600.087830,599.687104,1200.175660,1199.374207
14,334.926019,0.278538,1234,335.204557,334.647480,670.409113,669.294961


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-653.259050,651.870170
6,-380.622546,380.602700
7,-542.809255,543.516295
8,-896.321566,896.916172
9,-132.544504,130.971422
10,-16.251062,15.585133
11,-164.571983,165.279069
12,-618.112045,618.682526
13,-600.087830,599.687104
14,-335.204557,334.647480


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-653.259050,651.870170
6,-380.622546,380.602700
7,-542.809255,543.516295
8,-896.321566,896.916172
9,-132.544504,130.971422
10,-16.251062,15.585133
11,-164.571983,165.279069
12,-618.112045,618.682526
13,-600.087830,599.687104
14,-335.204557,334.647480


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-653.25904973,  651.87016952],
       [-380.62254621,  380.60269998],
       [-542.80925528,  543.51629521],
       [-896.32156566,  896.91617206],
       [-132.54450418,  130.97142229],
       [ -16.25106171,   15.58513322],
       [-164.57198305,  165.27906938],
       [-618.11204545,  618.68252565],
       [-600.08783017,  599.68710351],
       [-335.20455656,  334.6474805 ]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1304.518099,-1303.740339
6,759.245092,-761.205400
7,1083.618511,-1087.032590
8,1790.643131,-1793.832344
9,263.089008,-261.942845
10,30.502123,-31.170266
11,327.143966,-330.558139
12,1234.224091,-1237.365051
13,1198.175660,-1199.374207
14,668.409113,-669.294961


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
